In [1]:
import sys
sys.path.append('..')
import numpy as np

In [34]:
from qiskit.circuit.library import QFT
from qiskit.converters import circuit_to_dag
from qiskit.dagcircuit import DAGCircuit, DAGOpNode
from qiskit import transpile
from coupling_map.coupling_map import one_dim_couping

# 测试电路
n_qubits = 10
basis_gates = ['rx', 'ry', 'cz']
qc = QFT(n_qubits)
qc = transpile(qc, basis_gates=basis_gates, coupling_map=one_dim_couping(n_qubits))
dag = circuit_to_dag(qc)

In [36]:
from dataset.high import get_dataset

circuits = [data['circuit'] for data in get_dataset(2, 11)]
dags = [circuit_to_dag(transpile(circuit, basis_gates=basis_gates, coupling_map=one_dim_couping(circuit.num_qubits))) for circuit in circuits]
print(dags)

[<qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc482c9780>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc48525bd0>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc4855b340>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc48190310>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc48132fe0>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc484a5cf0>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc48492020>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc48491e40>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fd05011e6b0>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc484893c0>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc2bfa5990>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc2bfad9c0>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc2bfa6200>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object at 0x7fcc48111570>, <qiskit.dagcircuit.dagcircuit.DAGCircuit object

In [45]:
def count_2pattern(dags: list[DAGCircuit]):
    patterns = {}
    for dag in dags:
        for edge in dag.edges():
            # 遍历边
            if isinstance(edge[0], DAGOpNode) and isinstance(edge[1], DAGOpNode):
                pattern = []
                for node in edge:
                    if isinstance(node, DAGOpNode):
                        if node.op.name == 'cz':
                            pattern.append(('cz', ))
                        elif node.op.name in basis_gates:
                            pattern.append((node.op.name, float(node.op.params[0])))
                pattern = tuple(pattern)
                if pattern in patterns:
                    patterns[pattern] += 1
                else:
                    patterns[pattern] = 1
    return patterns

patterns = count_2pattern(dags)
patterns = {k: v for k, v in patterns.items() if v >= 8}
patterns

{(('rx', -2.3561944901923457), ('cz',)): 34,
 (('rx', 0.7853981633974474), ('ry', -1.5707963267948966)): 45,
 (('ry', -1.5707963267948966), ('cz',)): 304,
 (('cz',), ('cz',)): 2951,
 (('cz',), ('rx', -1.5707963267948966)): 126,
 (('rx', -1.5707963267948966), ('cz',)): 435,
 (('cz',), ('ry', 1.5707963267948966)): 7587,
 (('ry', 1.5707963267948966), ('rx', 3.141592653589793)): 7833,
 (('rx', -2.7488935718910685), ('ry', -1.5707963267948968)): 22,
 (('ry', -1.5707963267948968), ('cz',)): 38,
 (('rx', -0.7853981633974492), ('cz',)): 263,
 (('cz',), ('rx', 0.3926990816987237)): 103,
 (('rx', 0.3926990816987237), ('cz',)): 107,
 (('cz',), ('rx', -3.141592653589793)): 147,
 (('rx', -3.141592653589793), ('ry', -1.5707963267948963)): 29,
 (('rx', 3.141592653589793),): 237,
 (('ry', 1.5707963267948966),): 118,
 (('rx', 1.5707963267948966),): 19,
 (('rx', 1.5707963267948966), ('ry', 0.5890486225480862)): 8,
 (('ry', 0.5890486225480862), ('rx', -1.5707963267948966)): 8,
 (('rx', 3.141592653589793)

In [49]:
qubits_list = [5, 10, 15, 20, 25, 30, 35, 50]

circuit_name = []
run_time = []
import time
import random
from common.All_pulse_generation import wave_construction

wx = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
wcz = np.pad(wave_construction('cz').real, (0, 80), 'constant', constant_values=(0, 0))

for n_qubits in qubits_list:
    dataset = get_dataset(n_qubits, n_qubits + 1)
    circuit_name.extend([data['id'] for data in dataset])
    circuits = [data['circuit'] for data in dataset]
    dags = [circuit_to_dag(transpile(circuit, basis_gates=basis_gates, coupling_map=one_dim_couping(circuit.num_qubits), optimization_level=0)) for circuit in circuits]
    
    for dag in dags:
        t = time.time()
        pulse_array = np.zeros((n_qubits, dag.depth() * 200))
        qubits_idx = [0 for _ in range(n_qubits)]
        for edge in dag.edges():
            if isinstance(edge[0], DAGOpNode) and isinstance(edge[1], DAGOpNode):
                cur = []
                for node in edge:
                    if isinstance(node, DAGOpNode):
                        if node.op.name == 'cz':
                            cur.append(('cz', ))
                        elif node.op.name in basis_gates:
                            cur.append((node.op.name, float(node.op.params[0])))
                cur = tuple(cur)
                if cur in patterns or (cur[1], cur[0]) in patterns:
                    pulse_array[0][0: 200] = wcz
                    pulse_array[1][200: 400] = wx
                    if random.random() > 0.5:
                        pulse_array[2][400: 600] = wcz
                else:
                    pulse_array[0][0: 200] = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
                    pulse_array[1][200: 400] = np.pad(wave_construction('cz').real, (0, 80), 'constant', constant_values=(0, 0))
                    if random.random() > 0.5:
                        pulse_array[2][400: 600] = np.pad(wave_construction('cz').real, (0, 80), 'constant', constant_values=(0, 0))
            else:
                pulse_array[3][0: 200] = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
        run_time.append(time.time() - t)
    print(f'finish {n_qubits}')

import pandas as pd

df = pd.DataFrame([circuit_name, run_time])
df.to_excel('result_graph.xlsx', header=None)

finish 5
finish 10
finish 15
finish 20
finish 25
finish 30
finish 35
finish 50
